<p style="text-align: center"><img src="https://gitlab.aicrowd.com/aicrowd/assets/-/raw/master/challenges/clock-decomposition/notebook-banner.jpg?inline=false" alt="Drawing" style="height: 400px;"/></p>

# What is the notebook about?

The challenge is to use the features extracted from the Clock Drawing Test to build an automated and algorithm to predict whether each participant is one of three phases:

1)    Pre-Alzheimer’s (Early Warning)
2)    Post-Alzheimer’s (Detection)
3)    Normal (Not an Alzheimer’s patient)

In machine learning terms: this is a 3-class classification task.

# How to use this notebook? 📝

<p style="text-align: center"><img src="https://gitlab.aicrowd.com/aicrowd/assets/-/raw/master/notebook/aicrowd_notebook_submission_flow.png?inline=false" alt="notebook overview" style="width: 650px;"/></p>

- **Update the config parameters**. You can define the common variables here

Variable | Description
--- | ---
`AICROWD_DATASET_PATH` | Path to the file containing test data (The data will be available at `/ds_shared_drive/` on aridhia workspace). This should be an absolute path.
`AICROWD_PREDICTIONS_PATH` | Path to write the output to.
`AICROWD_ASSETS_DIR` | In case your notebook needs additional files (like model weights, etc.,), you can add them to a directory and specify the path to the directory here (please specify relative path). The contents of this directory will be sent to AIcrowd for evaluation.
`AICROWD_API_KEY` | In order to submit your code to AIcrowd, you need to provide your account's API key. This key is available at https://www.aicrowd.com/participants/me

- **Installing packages**. Please use the [Install packages 🗃](#install-packages-) section to install the packages
- **Training your models**. All the code within the [Training phase ⚙️](#training-phase-) section will be skipped during evaluation. **Please make sure to save your model weights in the assets directory and load them in the predictions phase section** 

# Setup AIcrowd Utilities 🛠

We use this to bundle the files for submission and create a submission on AIcrowd. Do not edit this block.

In [1]:
!pip install -q -U aicrowd-cli

In [2]:
%load_ext aicrowd.magic

# AIcrowd Runtime Configuration 🧷

Define configuration parameters. Please include any files needed for the notebook to run under `ASSETS_DIR`. We will copy the contents of this directory to your final submission file 🙂

The dataset is available under `/ds_shared_drive` on the workspace.

In [3]:
import os

# Please use the absolute for the location of the dataset.
# Or you can use relative path with `os.getcwd() + "test_data/validation.csv"`
AICROWD_DATASET_PATH = os.getenv("DATASET_PATH", "/ds_shared_drive/validation.csv")
AICROWD_PREDICTIONS_PATH = os.getenv("PREDICTIONS_PATH", "predictions.csv")
AICROWD_ASSETS_DIR = "assets"


# Install packages 🗃

Please add all pacakage installations in this section

In [4]:
!pip install -q numpy pandas

In [5]:
!pip install -q -U fastcore fastai

In [6]:
!pip install -q torch

In [7]:
!pip install -q optuna

In [8]:
!pip install -q seaborn lightgbm scikit-learn

# Define preprocessing code 💻

The code that is common between the training and the prediction sections should be defined here. During evaluation, we completely skip the training section. Please make sure to add any common logic between the training and prediction sections here.

### Import common packages

Please import packages that are common for training and prediction phases here.

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import NearestCentroid, LocalOutlierFactor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from fastai.tabular.all import *
from sklearn.utils import compute_class_weight
import torch
import optuna
import joblib

In [10]:
train_data = pd.read_csv(os.getenv("DATASET_PATH", "/ds_shared_drive/train.csv"))
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32777 entries, 0 to 32776
Columns: 122 entries, row_id to diagnosis
dtypes: float64(111), int64(8), object(3)
memory usage: 30.5+ MB


# Training phase ⚙️

You can define your training code here. This sections will be skipped during evaluation.

In [11]:
# 

## Load training data

In [12]:
# class_wts = compute_class_weight(class_weight='balanced', 
#                                 classes=['normal','post_alzheimer', 'pre_alzheimer'],y=train_data.diagnosis)
# class_wts = torch.from_numpy(class_wts)
class_wts = torch.FloatTensor([0.3501,  9.5088, 26.0135])
# class_wts

In [13]:
#train_data = train_data[train_data.diagnosis.isin(['post_alzheimer','pre_alzheimer'])]\
# .append(train_data[train_data.diagnosis.isin(['normal'])].sample(6000))
#train_data.info()

In [14]:
'''interaction_generator = PolynomialFeatures(degree=2, interaction_only=True)
train_interactions = interaction_generator.fit_transform(train_data.fillna(0).drop(columns=['row_id', 
                                                                                  'intersection_pos_rel_centre', 
                                                                                  'diagnosis',
                                                                                  'actual_hour_digit',
                                                                                  'actual_minute_digit']))
train_interactions = pd.DataFrame(train_interactions)
train_interactions.columns = interaction_generator.get_feature_names()
train_interactions.info()
train_data = pd.concat([train_data, train_interactions], axis=1)
train_data.info()'''

"interaction_generator = PolynomialFeatures(degree=2, interaction_only=True)\ntrain_interactions = interaction_generator.fit_transform(train_data.fillna(0).drop(columns=['row_id', \n                                                                                  'intersection_pos_rel_centre', \n                                                                                  'diagnosis',\n                                                                                  'actual_hour_digit',\n                                                                                  'actual_minute_digit']))\ntrain_interactions = pd.DataFrame(train_interactions)\ntrain_interactions.columns = interaction_generator.get_feature_names()\ntrain_interactions.info()\ntrain_data = pd.concat([train_data, train_interactions], axis=1)\ntrain_data.info()"

In [15]:
dep_var = 'diagnosis'
cont_var = train_data.drop(columns=['row_id', 'intersection_pos_rel_centre', 
                                    'diagnosis',
                                    'actual_hour_digit',
                                    'actual_minute_digit'
                                   ]).columns.tolist() 
cat_var = ['intersection_pos_rel_centre']

procs = [FillMissing, Categorify, Normalize]

kfold = KFold(n_splits=5, random_state=42, shuffle=True)

val_data = pd.read_csv(os.getenv("DATASET_PATH", "/ds_shared_drive/validation.csv"))
val_actuals = pd.read_csv(os.getenv("DATASET_PATH", "/ds_shared_drive/validation_ground_truth.csv"))
val_merge = val_data.merge(val_actuals, on=['row_id'])
train_data_full = train_data.append(val_merge[train_data.columns])


target_col = "diagnosis"
target_values = ["normal", "post_alzheimer", "pre_alzheimer"]
seed=2021
'''df_pos = train_data_full[train_data_full[target_col].isin(target_values[1:])]
nb_pos = df_pos.shape[0]
nb_neg = nb_pos
df_neg = train_data_full[train_data_full[target_col] == "normal"].sample(n=nb_neg, random_state=seed)
df_samples = pd.concat([df_pos, df_neg]).sample(frac=1).reset_index(drop=True)
train_data_full = df_samples.copy(deep=True)'''



train_data_full.shape, train_data.shape

((33139, 122), (32777, 122))

## Train your model

In [16]:
'''learners = []
loss_func = CrossEntropyLossFlat(weight=class_wts)
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_data)):
    print(fold)
    splits = RandomSplitter(valid_pct=0.01)(range_of(train_data.iloc[train_ids]))
    # splits = IndexSplitter(test_ids.tolist())
    to = TabularPandas(train_data, procs=procs, cat_names=cat_var, 
                   cont_names=cont_var, y_names=dep_var, splits=splits)
    dls = to.dataloaders(bs=64)
    learner = tabular_learner(dls, metrics=accuracy, layers=[512,256], 
                              opt_func=RAdam, 
                              loss_func=loss_func
                             )
    learner.fit_one_cycle(5)
    
    dl_val = learner.dls.test_dl(val_merge)
    preds = learner.get_preds(dl=dl_val)
    # print(preds[0].numpy().shape)
    per_ = log_loss(val_merge['diagnosis'].values, preds[0].numpy())
    
    learners.append([fold, learner, per_])'''

"learners = []\nloss_func = CrossEntropyLossFlat(weight=class_wts)\nfor fold, (train_ids, test_ids) in enumerate(kfold.split(train_data)):\n    print(fold)\n    splits = RandomSplitter(valid_pct=0.01)(range_of(train_data.iloc[train_ids]))\n    # splits = IndexSplitter(test_ids.tolist())\n    to = TabularPandas(train_data, procs=procs, cat_names=cat_var, \n                   cont_names=cont_var, y_names=dep_var, splits=splits)\n    dls = to.dataloaders(bs=64)\n    learner = tabular_learner(dls, metrics=accuracy, layers=[512,256], \n                              opt_func=RAdam, \n                              loss_func=loss_func\n                             )\n    learner.fit_one_cycle(5)\n    \n    dl_val = learner.dls.test_dl(val_merge)\n    preds = learner.get_preds(dl=dl_val)\n    # print(preds[0].numpy().shape)\n    per_ = log_loss(val_merge['diagnosis'].values, preds[0].numpy())\n    \n    learners.append([fold, learner, per_])"

In [17]:
nc = NearestCentroid()
nc.fit(train_data_full.drop(columns=['row_id',
                                     'intersection_pos_rel_centre',
                                     'diagnosis']).fillna(0), train_data_full['diagnosis'])
train_data_full['nc'] = nc.predict(train_data_full.drop(columns=['row_id',
                                     'intersection_pos_rel_centre',
                                     'diagnosis']).fillna(0)).reshape((-1,1))
nc_num = {'nc':{'normal':1, 'pre_alzheimer':2, 'post_alzheimer':3}}

train_data_full.replace(nc_num, inplace=True)

#train_data_full.re
#lo = LocalOutlierFactor(n_neighbors=5, novelty=True)
#lo.fit(X_train_final, y_train)

'''X_train_final = np.concatenate([X_train_final, nc.predict(X_train_final).reshape((-1,1)),
                                #lo.predict(X_train_final).reshape((-1,1))
                               ], axis=1)
X_test_final = np.concatenate([X_test_final, nc.predict(X_test_final).reshape((-1,1)), 
                               #lo.predict(X_train_final).reshape((-1,1))
                              ], axis=1)'''

'X_train_final = np.concatenate([X_train_final, nc.predict(X_train_final).reshape((-1,1)),\n                                #lo.predict(X_train_final).reshape((-1,1))\n                               ], axis=1)\nX_test_final = np.concatenate([X_test_final, nc.predict(X_test_final).reshape((-1,1)), \n                               #lo.predict(X_train_final).reshape((-1,1))\n                              ], axis=1)'

In [18]:
poly_feats = PolynomialFeatures(degree=2, interaction_only=True)
X_train_int = poly_feats.fit_transform(train_data_full.drop(columns=['row_id',
                                                                     'intersection_pos_rel_centre',
                                                                     'diagnosis']).fillna(0))
X_train_int.shape

(33139, 7261)

In [19]:
rf_poly = RandomForestClassifier(n_estimators=200)
rf_poly.fit(X_train_int, train_data_full['diagnosis'])
importances = rf_poly.feature_importances_
importances

array([0.00000000e+00, 7.65892722e-04, 1.74924459e-05, ...,
       9.64126646e-05, 8.88860627e-05, 1.07888207e-05])

In [20]:
int_index = (-importances).argsort()[:200]
X_poly = X_train_int[:,int_index]
df_poly = pd.DataFrame(X_poly)
df_poly.columns = [poly_feats.get_feature_names()[i] for i in int_index]
df_poly.shape

(33139, 200)

In [21]:
train_data_full = pd.concat([train_data_full.reset_index(drop=True), df_poly.reset_index(drop=True)], axis=1)
train_data_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33139 entries, 0 to 33138
Columns: 323 entries, row_id to x101
dtypes: float64(311), int64(9), object(3)
memory usage: 81.7+ MB


In [22]:
val_merge['nc'] = nc.predict(val_merge.drop(columns=['row_id',
                                     'intersection_pos_rel_centre',
                                     'diagnosis']).fillna(0)).reshape((-1,1))

val_merge.replace(nc_num, inplace=True)

X_val_int = poly_feats.transform(val_merge.drop(columns=['row_id',
                                                         'intersection_pos_rel_centre',
                                                         'diagnosis']).fillna(0))

X_poly_val = X_val_int[:,int_index]
df_poly_val = pd.DataFrame(X_poly_val)
df_poly_val.columns = [poly_feats.get_feature_names()[i] for i in int_index]
val_merge = pd.concat([val_merge, df_poly_val], axis=1)
val_merge.shape

(362, 323)

In [23]:
train_data_all_in = train_data_full.append(val_merge[train_data_full.columns])
train_data_full, val_merge =  train_test_split(train_data_all_in, test_size=0.2, random_state=42)
print(train_data_full.shape, val_merge.shape)
train_data_full.diagnosis.value_counts(), val_merge.diagnosis.value_counts()

(26800, 323) (6701, 323)


(normal            25399
 post_alzheimer     1025
 pre_alzheimer       376
 Name: diagnosis, dtype: int64,
 normal            6349
 post_alzheimer     248
 pre_alzheimer      104
 Name: diagnosis, dtype: int64)

In [24]:
train_data_full = train_data_full.fillna(0)
val_merge = val_merge.fillna(0)

In [25]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_data_full))
to = TabularPandas(train_data_full, procs=procs, cat_names=cat_var, 
                   cont_names=cont_var+df_poly.columns.tolist(), y_names=dep_var, 
                   splits=splits)
loss_func = CrossEntropyLossFlat(weight=class_wts)
    

def objective(trial: optuna.Trial):
    num_layers = trial.suggest_int('n_layers', 1, 5)
    num_epochs = trial.suggest_int('n_epochs', 3, 10)
    bs_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    layers, ps = [],[]
    for i in range(num_layers-1):
        num_units = trial.suggest_categorical(f'num_units_layer_{i}', [800, 900, 1000, 1100, 1200])
        p = trial.suggest_discrete_uniform(f'dropout_p_layer_{i}',0,1,0.05)
        layers.append(num_units)
        ps.append(p)
        
    emb_drop = trial.suggest_discrete_uniform('emb_drop',0,1,0.05)
    
    dls = to.dataloaders(bs=bs_size)
    tc = tabular_config(ps=ps, embed_p=emb_drop)
    learner = tabular_learner(dls, metrics=accuracy, layers = layers,  
                              opt_func=RAdam, 
                              # loss_func=loss_func
                             )
    learner.fit_one_cycle(num_epochs)
    
    dl_val = learner.dls.test_dl(val_merge)
    preds = learner.get_preds(dl=dl_val)
    # preds = learner.get_preds(Dataset.T)
    per_ = log_loss(val_merge['diagnosis'].values, preds[0].numpy())
    
    return per_

study = optuna.create_study()
study.optimize(objective)
best_trial = study.best_trial


[I 2021-05-29 06:01:30,383] A new study created in memory with name: no-name-1b0c2a17-c28d-4a3b-ad21-ca01eb69c79d


epoch,train_loss,valid_loss,accuracy,time
0,0.267430,0.219595,0.946269,00:30
1,0.230794,0.201248,0.947575,00:30
2,0.165231,0.213454,0.947575,00:30
3,0.203344,0.198347,0.947575,00:31
4,0.197803,0.207097,0.944776,00:31
5,0.200628,0.195479,0.946642,00:31
6,0.157963,0.201572,0.946082,00:31
7,0.175441,0.198999,0.945709,00:30


[I 2021-05-29 06:05:39,777] Trial 0 finished with value: 0.19702312785620063 and parameters: {'n_layers': 4, 'n_epochs': 8, 'batch_size': 32, 'num_units_layer_0': 1100, 'dropout_p_layer_0': 0.9, 'num_units_layer_1': 1000, 'dropout_p_layer_1': 0.35000000000000003, 'num_units_layer_2': 900, 'dropout_p_layer_2': 0.35000000000000003, 'emb_drop': 0.30000000000000004}. Best is trial 0 with value: 0.19702312785620063.


epoch,train_loss,valid_loss,accuracy,time
0,0.352873,0.222508,0.946269,00:18
1,0.234668,0.212265,0.947575,00:19
2,0.200165,0.206344,0.943843,00:19
3,0.193690,0.192703,0.947761,00:19
4,0.173569,0.194059,0.944403,00:19


[I 2021-05-29 06:07:17,526] Trial 1 finished with value: 0.1934392379256272 and parameters: {'n_layers': 5, 'n_epochs': 5, 'batch_size': 64, 'num_units_layer_0': 900, 'dropout_p_layer_0': 0.7000000000000001, 'num_units_layer_1': 800, 'dropout_p_layer_1': 0.4, 'num_units_layer_2': 800, 'dropout_p_layer_2': 0.4, 'num_units_layer_3': 1100, 'dropout_p_layer_3': 0.65, 'emb_drop': 1.0}. Best is trial 1 with value: 0.1934392379256272.


...

[I 2021-05-30 00:05:51,919] Trial 416 finished with value: 0.19175948162697462 and parameters: {'n_layers': 4, 'n_epochs': 3, 'batch_size': 32, 'num_units_layer_0': 1200, 'dropout_p_layer_0': 0.8, 'num_units_layer_1': 900, 'dropout_p_layer_1': 0.5, 'num_units_layer_2': 900, 'dropout_p_layer_2': 0.7000000000000001, 'emb_drop': 0.5}. Best is trial 71 with value: 0.18762676680330917.


epoch,train_loss,valid_loss,accuracy,time
0,0.338808,0.261684,0.947575,00:24
1,0.215914,0.221199,0.938060,00:28
2,0.193448,0.199175,0.946828,00:49
3,0.175832,0.195035,0.944776,00:58


[I 2021-05-30 00:08:36,534] Trial 417 finished with value: 0.1913369953130761 and parameters: {'n_layers': 4, 'n_epochs': 4, 'batch_size': 64, 'num_units_layer_0': 800, 'dropout_p_layer_0': 0.9, 'num_units_layer_1': 900, 'dropout_p_layer_1': 0.55, 'num_units_layer_2': 900, 'dropout_p_layer_2': 0.65, 'emb_drop': 0.6000000000000001}. Best is trial 71 with value: 0.18762676680330917.


epoch,train_loss,valid_loss,accuracy,time
0,0.196458,0.231857,0.936567,01:19
1,0.202027,0.206243,0.944963,01:15


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ <ipython-input-25-3fc695af602d>:37 in <module>                                            │
│                                                                                           │
│ /home/desktop0/conda/lib/python3.8/site-packages/optuna/study.py:400 in optimize          │
│                                                                                           │
│    397 │   │   │   │   FutureWarning,                                                     │
│    398 │   │   │   )                                                                      │
│    399 │   │                                                                              │
│ ❱  400 │   │   _optimize(                                                                 │
│    401 │   │   │   study=self,                                                            │
│    402 │   │   │   func=func,                                                             │
│    403 │   │   │   n_trials=n_trials,                                                     │
│                                                                                           │
│ /home/desktop0/conda/lib/python3.8/site-packages/optuna/_optimize.py:66 in _optimize      │
│                                                                                           │
│    63 │                                                                                   │
│    64 │   try:                                                                            │
│    65 │   │   if n_jobs == 1:                                                             │
│ ❱  66 │   │   │   _optimize_sequential(                                                   │
│    67 │   │   │   │   study,                                                              │
│    68 │   │   │   │   func,                                                               │
│    69 │   │   │   │   n_trials,                                                           │
│                                                                                           │
│ /home/desktop0/conda/lib/python3.8/site-packages/optuna/_optimize.py:163 in               │
│ _optimize_sequential                                                                      │
│                                                                                           │
│   160 │   │   │   │   break                                                               │
│   161 │   │                                                                               │
│   162 │   │   try:                                                                        │
│ ❱ 163 │   │   │   trial = _run_trial(study, func, catch)                                  │
│   164 │   │   except Exception:                                                           │
│   165 │   │   │   raise                                                                   │
│   166 │   │   finally:                                                                    │
│                                                                                           │
│ /home/desktop0/conda/lib/python3.8/site-packages/optuna/_optimize.py:217 in _run_trial    │
│                                                                                           │
│   214 │   │   thread.start()                                                              │
│   215 │                                                                                   │
│   216 │   try:                                                                            │
│ ❱ 217 │   │   value_or_values = func(trial)                                               │
│   218 │   except exceptions.TrialPruned as e:                                             │
│   219 │   │   # TODO(mamu): Handle multi-objective cases.                                 │
│   220 │   │   state = TrialState.PRUNED           

In [ ]:
# best_trial

In [ ]:
#  {'n_layers': 4, 'n_epochs': 3, 'batch_size': 32, 'num_units_layer_0': 1000, 'dropout_p_layer_0': 0.9, 'num_units_layer_1': 1100, 'dropout_p_layer_1': 0.35000000000000003, 'num_units_layer_2': 800, 'dropout_p_layer_2': 0.6000000000000001, 'emb_drop': 0.5}. 


In [26]:
#  Best is trial 71 with value: 0.18762676680330917.
best_parameters = {'n_layers': 4, 
                   'n_epochs': 3, 
                   'batch_size': 32, 
                   'num_units_layer_0': 1000, 
                   'dropout_p_layer_0': 0.9, 
                   'num_units_layer_1': 1100, 
                   'dropout_p_layer_1': 0.35000000000000003, 
                   'num_units_layer_2': 800, 
                   'dropout_p_layer_2': 0.6000000000000001, 
                   #'num_units_layer_3': 1100, 
                   #'dropout_p_layer_3': 0.05, 
                   'emb_drop': 0.5}

In [27]:
# splits = RandomSplitter(valid_pct=0.01)(range_of(train_data_full))
splits = RandomSplitter(valid_pct=0.01)(range_of(train_data_all_in))
to = TabularPandas(train_data_all_in, procs=procs, cat_names=cat_var, 
                   cont_names=cont_var, y_names=dep_var, 
                   splits=splits)
dls = to.dataloaders(bs=best_parameters['batch_size'])

tc = tabular_config(
    ps=[best_parameters['dropout_p_layer_0'],
                        best_parameters['dropout_p_layer_1'], 
                        best_parameters['dropout_p_layer_2'], 
                        #best_parameters['dropout_p_layer_3']
       ], 
                    embed_p=best_parameters['emb_drop']
                   )
learner_best = tabular_learner(dls, metrics=accuracy, 
                               layers = [best_parameters['num_units_layer_0'],
                                                                best_parameters['num_units_layer_1'],
                                                                best_parameters['num_units_layer_2'],
                                                                #best_parameters['num_units_layer_3']
                                        ],  
                              opt_func=RAdam, 
                              # loss_func=loss_func
                             )
learner_best.fit_one_cycle(best_parameters['n_epochs'])

epoch,train_loss,valid_loss,accuracy,time
0,0.225536,0.282353,0.919403,02:24
1,0.180918,0.222185,0.940298,03:20
2,0.181397,0.224406,0.943284,02:50


## Save your trained model

In [28]:
# learner.export(AICROWD_ASSETS_DIR+'/fastai_tabular.mdl')
#for i in range(len(learners)):
#    learners[i][1].export(AICROWD_ASSETS_DIR+'/fastai_tabular'+str(i)+'.mdl')
# learners[best_model][1].export(AICROWD_ASSETS_DIR+'/fastai_tabular.mdl')
learner_best.export(AICROWD_ASSETS_DIR+'/fastai_tabular_4.mdl')

poly_filename = f'{AICROWD_ASSETS_DIR}/poly_feats.pkl'
joblib.dump(poly_feats, poly_filename)

nc_filename = f'{AICROWD_ASSETS_DIR}/nc_model.pkl'
joblib.dump(nc, nc_filename)

nc_num_filename  = f'{AICROWD_ASSETS_DIR}/nc_num.pkl'
joblib.dump(nc_num, nc_num_filename)

meta = {
    "inter_index": int_index
}
meta_filename = f'{AICROWD_ASSETS_DIR}/model_lgb_meta.pkl'
joblib.dump(meta, meta_filename)

['assets/model_lgb_meta.pkl']

# Prediction phase 🔎

Please make sure to save the weights from the training section in your assets directory and load them in this section

In [29]:
# model = load_model_from_assets_dir(AIAICROWD_ASSETS_DIR)
# learners = [load_learner(AICROWD_ASSETS_DIR+'/fastai_tabular'+str(i)+'.mdl') for i in range(5)]
# len(learners)
learner = load_learner(AICROWD_ASSETS_DIR+'/fastai_tabular_4.mdl')

meta_filename = f'{AICROWD_ASSETS_DIR}/model_lgb_meta.pkl'
meta = joblib.load(meta_filename)
print(meta.keys())

poly_filename = f'{AICROWD_ASSETS_DIR}/poly_feats.pkl'
poly_feats = joblib.load(poly_filename)

nc_filename = f'{AICROWD_ASSETS_DIR}/nc_model.pkl'
nc = joblib.load(nc_filename)

nc_num_filename  = f'{AICROWD_ASSETS_DIR}/nc_num.pkl'
nc_num = joblib.load(nc_num_filename)

int_index = meta['inter_index']

dict_keys(['inter_index'])


## Load test data

In [30]:
test_data = pd.read_csv(AICROWD_DATASET_PATH)
test_data.head()

,row_id,number_of_digits,missing_digit_1,missing_digit_2,missing_digit_3,missing_digit_4,missing_digit_5,missing_digit_6,missing_digit_7,missing_digit_8,...,top_area_perc,bottom_area_perc,left_area_perc,right_area_perc,hor_count,vert_count,eleven_ten_error,other_error,time_diff,centre_dot_detect
0,LA9JQ1JZMJ9D2MBZV,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.500272,0.499368,0.553194,0.446447,0,0,0,1,NaN,NaN
1,PSSRCWAPTAG72A1NT,6.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.572472,0.427196,0.496352,0.503273,0,1,0,1,NaN,NaN
2,GCTODIZJB42VCBZRZ,11.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.494076,0.505583,0.503047,0.496615,1,0,0,0,0.0,0.0
3,7YMVQGV1CDB1WZFNE,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.555033,0.444633,0.580023,0.419575,0,1,0,1,NaN,NaN
4,PHEQC6DV3LTFJYIJU,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.603666,0.395976,0.494990,0.504604,0,0,0,1,150.0,0.0


## Generate predictions

In [31]:
test_data['nc'] = nc.predict(test_data.drop(columns=['row_id',
                                     'intersection_pos_rel_centre']).fillna(0)).reshape((-1,1))

test_data.replace(nc_num, inplace=True)


X_test_int = poly_feats.transform(test_data.drop(columns=['row_id',
                                                         'intersection_pos_rel_centre']).fillna(0))
X_poly_test = X_test_int[:,int_index]
df_poly_test = pd.DataFrame(X_poly_test)
df_poly_test.columns = [poly_feats.get_feature_names()[i] for i in int_index]
test_data = pd.concat([test_data, df_poly_test], axis=1)
test_data.shape

(362, 322)

In [32]:
'''pred_final = 0.0
for learner in learners:
    dl = learner.dls.test_dl(test_data)
    preds = learner.get_preds(dl=dl)
    preds = preds[0].numpy()
    pred_final = pred_final + preds
pred_final = pred_final/len(learners)
preds = pred_final'''
dl = learner.dls.test_dl(test_data)
preds = learner.get_preds(dl=dl)
preds = preds[0].numpy()
# preds = np.round(preds,2)
preds.shape

(362, 3)

In [33]:
pred_1 = []
for pred in preds:
    if(pred.argmax()==0):
        pred = [0.6, 0.2, 0.2]
    elif(pred.argmax()==1):
        pred = [0.2, 0.6, 0.2]
    else:
        pred = [0.2, 0.2, 0.6]
    pred_1.append(pred)
preds = np.array(pred_1)
preds.shape

(362, 3)

In [34]:
predictions = {
    "row_id": test_data["row_id"].values,
    "normal_diagnosis_probability": preds[:,0],
    "post_alzheimer_diagnosis_probability": preds[:,1],
    "pre_alzheimer_diagnosis_probability": preds[:,2],
}

predictions_df = pd.DataFrame.from_dict(predictions)

## Save predictions 📨

In [35]:
predictions_df.to_csv(AICROWD_PREDICTIONS_PATH, index=False)

# Submit to AIcrowd 🚀

**NOTE: PLEASE SAVE THE NOTEBOOK BEFORE SUBMITTING IT (Ctrl + S)**

In [ ]:
!DATASET_PATH=$AICROWD_DATASET_PATH \
aicrowd notebook submit \
    --assets-dir $AICROWD_ASSETS_DIR \
    --challenge addi-alzheimers-detection-challenge